# Valuation of investment projects

# 1.1 Valuation of 2000 projects

In [2]:
import pandas as pd
import numpy as np

import numpy_financial as npf

pd.options.display.float_format = '{:,.2f}'.format

In [3]:
df = pd.read_csv("../../../data/raw/01_Data_OilCompany.csv")
df.head()

,Year 1,Year 2,Year 3,Year 4,Year 5
0,"203,726.00","279,969.00","465,303.00","200,445.00","96,319.00"
1,"263,845.00","272,439.00","2,192.00","199,901.00",0.00
2,"27,726.00","334,861.00","82,265.00","199,954.00","64,958.00"
3,"293,801.00","210,117.00","275,087.00","199,934.00",0.00
4,"297,319.00","242,569.00","464,097.00","200,598.00","141,285.00"


In [4]:
df.transpose().sum().max()

np.float64(1615419.0)

In [5]:
class financial_metrics:
    
    def calculate_npv(self, data, discount_rate):
        npv = npf.npv(discount_rate, data)
        return npv
    
    def calculate_irr(self, data):
        irr = npf.irr(data)
        return irr

In [6]:
def valuate_investment(data : pd.DataFrame, initial_investment : float, discount_rate : float) -> pd.DataFrame:
    financial_data = pd.DataFrame()
    
    initial_investment = pd.Series([initial_investment])
    
    fm = financial_metrics()
    
    for index, row in data.iterrows():
        cash_flows = row.values
        
        net_cash_flows = pd.concat([initial_investment, row]).values        

        npv = fm.calculate_npv(cash_flows, discount_rate)
        irr = fm.calculate_irr(net_cash_flows)
        
        financial_data.at[index, 'NPV'] = npv
        financial_data.at[index, 'IRR'] = irr
        
    return financial_data

In [7]:
initial_investment = -700_000
discount_rate = 0.03


financial_data = valuate_investment(df, initial_investment, discount_rate) 

In [8]:
financial_data

,NPV,IRR
0,"1,183,147.02",0.25
1,"713,352.79",0.02
2,"671,077.00",0.00
3,"940,061.90",0.16
4,"1,279,384.30",0.29
...,...,...
1995,"845,919.82",0.10
1996,"734,515.67",0.04
1997,"1,103,394.08",0.23
1998,"1,144,001.16",0.23


### Considerations

In this case sinse the initial outflow is equal to $700,000 the following question need considering this cash flow. We choose this outflow because we calculate some statistics based on the csv, then we found that the average sum of cash flow is $963,101, so if we use the outflow using in the class $2,000,000 the most projects will be rejected because the IRR is lower than cero.

For this reason we choose $700,000 as initial outflow.

However the discounted rate is the same that we use in the class 3% as the risk free rate.

### Questions

In [9]:
n = len(financial_data)

#### 1. question

What the probability that the IRR is over the Risk free rate?

In [10]:
x = financial_data[financial_data["IRR"] > discount_rate]["IRR"].count()
x/n

np.float64(0.793)

The probability is 79.3% that the IRR is over the risk free rate

#### 2. question

What the probability that the IRR is over the 35%?

In [11]:
q2_discount_rate = 0.35

x = financial_data[financial_data["IRR"] > q2_discount_rate]["IRR"].count()
x/n

np.float64(0.008)

Only the 0.8% the project will have a IRR over the 35%

#### 3. question

What the probability that the project value is over $2M?

In [12]:
wish_value = 2_000_000
x = financial_data[financial_data["NPV"] > wish_value]["NPV"].count()
x/n

np.float64(0.0)

There does not exist any project that the NPV is over $2M, so the probability is 0%

#### 4. question

What’s the probability that the IRR is between 10% and 20%?

In [13]:
q4_discount_rate_sup = 0.20
q4_discount_rate_inf = 0.10

x = financial_data[(financial_data["IRR"] > q4_discount_rate_inf) * (financial_data["IRR"] < q4_discount_rate_sup)]["IRR"].count()
x/n

np.float64(0.3245)

The probability that the project IRR is between 10% and 20% is 32.45%

# 1.2 adding the uncertainty factor on one project

### Exercise: stock with dividends and uncertain growth

We use a dividend model with terminal value (Gordon). The growth rate g is the stochastic variable, chosen at the start and kept constant over the horizon.

#### 1. Conditions
- Current stock price (S): $15.00
- Current dividend (D0): $0.40
- Discount rate (R): 3% per year
- Explicit horizon: 5 years, with terminal value using D_{6}/(R-g)
- Scenarios for g: 0.5% (p=0.30), 1.0% (p=0.50), 1.5% (p=0.20)


In [14]:
price = 15.00

# Dividend paid in the last period
dividend_0 = 0.40

discount_rate = 0.03
horizon_years = 5

scenarios = pd.DataFrame({
    "scenario": ["Low", "Base", "High"],
    "g": [0.005, 0.01, 0.015],
    "prob": [0.30, 0.50, 0.20],
})


def dividend_valuation(dividend_0, g, discount_rate, horizon_years):
    years = np.arange(1, horizon_years + 1)
    dividends = dividend_0 * (1 + g) ** years
    pv_dividends = dividends / (1 + discount_rate) ** years

    dividend_next = dividend_0 * (1 + g) ** (horizon_years + 1)
    terminal_value = dividend_next / (discount_rate - g)
    pv_terminal = terminal_value / (1 + discount_rate) ** horizon_years

    pv_total = pv_dividends.sum() + pv_terminal
    return pv_total, pv_terminal


rows = []
for _, row in scenarios.iterrows():
    pv_total, pv_terminal = dividend_valuation(
        dividend_0, row["g"], discount_rate, horizon_years
    )
    rows.append({
        "scenario": row["scenario"],
        "g": row["g"],
        "prob": row["prob"],
        "PV": pv_total,
        "NPV": pv_total - price,
        "PV_terminal": pv_terminal,
    })

summary = pd.DataFrame(rows)
summary["EV_PV"] = summary["PV"] * summary["prob"]
summary["EV_NPV"] = summary["NPV"] * summary["prob"]

summary


,scenario,g,prob,PV,NPV,PV_terminal,EV_PV,EV_NPV
0,Low,0.01,0.30,16.08,1.08,14.22,4.82,0.32
1,Base,0.01,0.50,20.20,5.20,18.31,10.10,2.60
2,High,0.01,0.20,27.07,12.07,25.15,5.41,2.41


#### 2. Decision tree and expected value


In [18]:
expected_pv = summary["EV_PV"].sum()
expected_npv = summary["EV_NPV"].sum()
break_even_price = expected_pv


def money(x):
    return f"${x:,.2f}"


def pct(x):
    return f"{x * 100:.1f}%"


decision = "Buy" if expected_npv > 0 else "Do not buy"

lines = [
    "Decision Summary",
    "-" * 60,
    f"Decision: {decision}",
    f"Price (S): {money(price)}",
    "",
    "Growth scenarios (g):",
]

for _, row in summary.iterrows():
    lines.append(
        f"  - {row['scenario']}: g={pct(row['g'])} | p={pct(row['prob'])}"
        f" | PV={money(row['PV'])} | NPV={money(row['NPV'])}"
    )

lines.extend([
    "",
    "Expected Values:",
    f"  - Expected PV: {money(expected_pv)}",
    f"  - Expected NPV: {money(expected_npv)}",
    f"  - Break-even price (NPV=0): {money(break_even_price)}",
])

print("".join(lines))

Decision Summary------------------------------------------------------------Decision: BuyPrice (S): $15.00Growth scenarios (g):  - Low: g=0.5% | p=30.0% | PV=$16.08 | NPV=$1.08  - Base: g=1.0% | p=50.0% | PV=$20.20 | NPV=$5.20  - High: g=1.5% | p=20.0% | PV=$27.07 | NPV=$12.07Expected Values:  - Expected PV: $20.34  - Expected NPV: $5.34  - Break-even price (NPV=0): $20.34


#### 3. When the decision changes

The decision changes if:
- The price S exceeds the break-even price (equal to expected PV); with S >= expected PV, the expected NPV is <= 0.
- The distribution of g shifts downward (lower average g or higher probability of the Low scenario).
- The discount rate rises; as R increases, the present value falls and can drop below S.
